### Training BERT on Labeled Endometriosis Dataset
This script trains, tests, and saves a BERT model on our labeled paragraphs (or posts) from the endometriosis dataset.

Additional resources for this code:


*   HuggingFace's docs on [fine-tuning a pre-trained model](https://huggingface.co/docs/transformers/training)
*   BERT for Humanist's [Fine-Tuning for Classification](https://colab.research.google.com/drive/19jDqa5D5XfxPU6NQef17BC07xQdRnaKU?usp=sharing) tutorial



In [1]:
# Mount the Google drive for access to files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Basic Python modules
import os
import re
from collections import defaultdict, Counter
import random
import pickle

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch

In [3]:
incerto_dir = '/content/drive/MyDrive/incerto-autore'
new_poems_dir = os.path.join(incerto_dir, 'data', 'poems')
poems_split_df = pd.read_csv(os.path.join(new_poems_dir, 'poems_split.csv'))
len(poems_split_df)

1176

In [4]:
!pip3 install transformers

In [5]:
# using DistilBERT for testing --> can switch to BERT once set up
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [6]:
# Choose the GPU we want to process this script
device_name = 'cuda'

# Choose the BERT model that we want to use (make sure to keep the cased/uncased consistent)
# model = 'dbmdz/bert-base-italian-xxl-uncased'
model = os.path.join(incerto_dir, 'contbertoldo-all', 'checkpoint')

# This is the maximum number of tokens in any document sent to BERT
max_length = 512

In [7]:
if 'contbertoldo' in model:
  finetuned_path = os.path.join(incerto_dir, 'output','finetuned-models', 'binary-class', 'bertoldo')
elif 'italian':
  finetuned_path = os.path.join(incerto_dir, 'output','finetuned-models', 'binary-class', 'bert-ita')
if not os.path.exists(finetuned_path):
  os.makedirs(finetuned_path)

#### Weighted trainer

In [8]:
from sklearn.utils.class_weight import compute_class_weight

### BERT setup

In [9]:
# load the encoder/tokenizer
tokenizer = BertTokenizer.from_pretrained(model)

In [10]:
# class for Torch dataset
class SCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,              # Increased from 3
    per_device_train_batch_size=16,
    per_device_eval_batch_size=20,
    learning_rate=2e-5,              # Lowered from 5e-5
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=100,                  # Added
    save_strategy='steps',           # Added
    save_steps=100,                  # Added
    load_best_model_at_end=True,     # Added
    metric_for_best_model='eval_f1', # Added
    greater_is_better=True,          # Added
)

In [12]:
# load pre-trained model
model = BertForSequenceClassification.from_pretrained(model).to(device_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/incerto-autore/contbertoldo-all/checkpoint and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Define a custom evaluation function (this could be changes to return accuracy metrics)
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    return {
        'accuracy': accuracy,
        'f1': f1,  # Make sure this is included
        'precision': precision,
        'recall': recall
    }

## Classification task setup

In [14]:
annotations_df = poems_split_df.loc[poems_split_df['author'] != 'UnknownAuthor']
len(annotations_df)

1116

In [17]:
for author in annotations_df['author'].unique():

  print(author)
  author_finetuned_path = os.path.join(finetuned_path, author)
  if not os.path.exists(author_finetuned_path):
    X = annotations_df['poem'].tolist()
    y = annotations_df['author'].map(lambda x: 1 if x==author else 0).tolist()
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.25)
    print('Y train', Counter(y_train))
    print('Y test', Counter(y_test))
    print(X_test[0:3])

    # Pass training/testing sentences to tokenizer, truncate them if over max length, and add padding (PAD tokens up to 512)
    train_encodings = tokenizer(X_train,  truncation=True, padding=True)
    test_encodings = tokenizer(X_test,  truncation=True, padding=True)

    # Combine encoded text and labels into a torch dataset object.
    train_dataset = SCDataset(train_encodings, y_train)
    test_dataset = SCDataset(test_encodings, y_test)

    # Calculate class weights from your training labels
    # Replace 'train_labels' with your actual training labels variable
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(y_train),
        y=y_train
    )

    print(f"Class weights: {class_weights}")  # This will show the calculated weights

    # Create custom trainer with class weights
    class WeightedTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get("logits")

            # Apply class weights
            loss_fct = torch.nn.CrossEntropyLoss(
                weight=torch.tensor(class_weights, dtype=torch.float32).to(logits.device)
            )
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

            return (loss, outputs) if return_outputs else loss

    # Create the trainer object based on what we've set up prior to this point! This combines our model, training_args, train_dataset and test_dataset, and custom evaluation function compute_metrics.
    trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    )

    # Fine-tune the model on our dataset/labels. The trainer object will periodically output the state of the model.
    trainer.train()

    # built in evaluation function
    trainer.evaluate()

    #save model
    model.save_pretrained(author_finetuned_path)

    print(Counter(y_test))

    predicted_labels = trainer.predict(test_dataset)
    actual_predicted_labels = predicted_labels.predictions.argmax(-1)

    class_report = classification_report(predicted_labels.label_ids.flatten(), actual_predicted_labels.flatten(), output_dict=True)
    print(classification_report(predicted_labels.label_ids.flatten(), actual_predicted_labels.flatten()))

    # New + simple save of classification report
    class_report_df = pd.DataFrame(class_report).transpose()
    # os.makedirs(author_finetuned_path)
    class_report_df.to_csv(os.path.join(author_finetuned_path, 'classification_report.csv'))

AntonGiacomoCorso
BartolomeoZacco
Y train Counter({0: 833, 1: 4})
Y test Counter({0: 278, 1: 1})
['volgeste a me la fronte humile e pia e nel contar il ben del ciel sovrano v affisaste a guardarmi e mi stendeste hor larghe or giunte l una e l altra mano e altre cose simili faceste ond io tolsi a sperar che del mio amore cautamente pietoso v accorgeste Quinci s accrebbe forte il mio dolore di non poter al gusto d ambo noi goder la vita in gioia e in dolzore Mesi e anni trascorsero dapoi ond a me variar convenne stile com ancor forse far convenne a voi hor vi miro non poco dissimile', 'Perch al viso d Amor portava insegna Mosse una pellegrina il mio cor vano Ch ogn altra mi parea d onor men degna E lei seguendo su per l herbe verdi Udi dir alta voce di lontano Ahi quanti passi per la selva perdi Alhor mi strinsi a l ombra d un bel faggio Tutto pensoso e rimirando intorno Vidi assai periglioso il mio viaggio E torna ndietro quasi a mezzo il giorno', 'V Amo Donna e di me sol perch io v amo

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.006000,0.367218,0.996416,0.994627,0.992844,0.996416
200,1.074900,0.413163,0.996416,0.994627,0.992844,0.996416


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Counter({0: 278, 1: 1})


...

Counter({0: 259, 1: 20})
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       259
           1       0.50      0.30      0.38        20

    accuracy                           0.93       279
   macro avg       0.72      0.64      0.67       279
weighted avg       0.92      0.93      0.92       279

ValerioSali
Y train Counter({0: 799, 1: 38})
Y test Counter({0: 267, 1: 12})
['vestigie espresse il bel luogo ritiene Cosi determinarono le stelle far quivi in dolci modi altrui palese quanto puon destinar e influir elle In questo avventuroso almo paese l ornamento del ciel si mostra in terra ch a farlo un paradiso in lui discese Di lieti colli adorno cerchio serra l infinita belta del vago piano dove Flora e Pomona alberga e erra Quasi per gradi su di mano in mano di fuor s ascende l poggio da le spalle sempre al salir piu facile e piu piano quinci in giu per soave e destro calle', 'Se l pensier che mi strugge Com e pungente e saldo Cosi v

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,1.043400,0.825012,0.939068,0.926917,0.915077,0.939068
200,0.759800,1.087270,0.888889,0.911387,0.942161,0.888889


Counter({0: 267, 1: 12})
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       267
           1       0.00      0.00      0.00        12

    accuracy                           0.94       279
   macro avg       0.48      0.49      0.48       279
weighted avg       0.92      0.94      0.93       279

VeronicaFranco
Y train Counter({0: 662, 1: 175})
Y test Counter({0: 221, 1: 58})
['Ahi che giace per man d invida morte Che l magnanimo ESTOR tolse di vita Chi devea non provar si tosto morte Anzi viver qua giu perpetua vita Benche meglio poria chiamarsi morte Questa nostra sia breve o lunga vita Poiche fin che non giunge il corpo morte Non puo l alma goder l eterna vita Ne doler ma piacer de quello morte Per cui l uom fa passaggio a miglior vita Senza mai piu temer futura morte Et via piu se chi pria ch esca di vita Ben oprando qual egli anzi la morte Resta al mondo per fama anchora in vita', 'Onde quella bontate E quest altra bellezza il 

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.432800,0.336757,0.870968,0.876365,0.888319,0.870968
200,0.164200,0.301248,0.935484,0.936265,0.937565,0.935484


Counter({0: 221, 1: 58})
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       221
           1       0.82      0.88      0.85        58

    accuracy                           0.94       279
   macro avg       0.90      0.91      0.90       279
weighted avg       0.94      0.94      0.94       279

